In [ ]:
for i in range(6):
    plt.figure(figsize=(20,6))
    plt.subplot(1, 2, 1)
    new_A_LSR = np.reshape(A_LSR[i],(21,21))
    sns.heatmap(new_A_LSR)
    plt.title('Retrieved Spatial Map A_LSR %d'%(i+1))
    
    plt.subplot(1, 2, 2)
    plt.plot(D_LSR.T[i])
    plt.title('Retrieved Time Couse D_LSR %d'%(i+1))

In [ ]:
import scipy.stats as stats

# Check the distribution for gamma t and gamma s
fig = plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
sns.distplot(gamma_t.flatten(), bins=25, label='temporal')
#std_norm_t = np.linspace(0 - 1.96*0.25, 0 + 1.96*0.25)
#plt.plot(std_norm_t, stats.norm.pdf(std_norm_t, 0, 0.25))
plt.savefig('/Volumes/E/MAST30034_Python/plots/Q1.4TN_dist.pdf')
plt.title('Histogram for Temporal Noise')

plt.subplot(1,2,2)
sns.distplot(gamma_s.flatten(), bins=25)
#std_norm_s = np.linspace(0 - 1.96*0.015, 0 + 1.96*0.015)
#plt.plot(std_norm_s, stats.norm.pdf(std_norm_s, 0, 0.015))
plt.title('Histogram for Spatial Noise')
plt.savefig('/Volumes/E/MAST30034_Python/plots/Q1.4SN_dist.pdf')
plt.show()

In [4]:
import numpy as np

(np.repeat(0, 6)).shape

(6,)

In [ ]:
# Recalculate D_LR and A_LR
# select the rho value with the smallest MSE

rho = 0
step = 1/(LA.norm(std_TC @ std_TC.T)*1.1)
thr = rho *N *step
Ao = np.zeros(nsrcs)
A = np.zeros(nsrcs)
A_LR = np.zeros((nsrcs, x1*x2)) 


gamma_t = np.random.normal(0, sqrt(0.25), size=(240,6))
gamma_s = np.random.normal(0, sqrt(0.015), size=(6,441))
X = (std_TC + gamma_t) @ (SM + gamma_s)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Modified R code for LASSO Regression
for k in range(x1*x2):
    A = Ao + step *(std_TC.T @ (X[:,k] - (std_TC @ Ao)))
    A_sign = np.sign(A)
    A = (1/(1+thr))*(A_sign*(np.maximum(np.zeros(nsrcs).T, (np.abs(A)-thr))))
    
    for j in range(10): 
        Ao = A
        A = Ao + step *(std_TC.T @ (X[:,k] - (std_TC @ Ao)))
        A_sign = np.sign(A)
        A = (1/(1+thr))*(A_sign*(np.maximum(np.zeros(nsrcs).T, (np.abs(A)-thr))))
        
    A_LR[:,k] = A.flatten()     
D_LR = X @ A_LR.T

#m = np.sum(np.square(X - D_LR @ A_LR)/(N*V))
m = np.square((LA.norm((X - D_LR @ A_LR), ord = 'fro')))/ (N *V)
m

In [ ]:
# Plot D_PCR and A_PCR side by side

plt.figure(figsize=(18,10))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.scatter(D_PCR[i,:], A_PCR[:,i])
    plt.xlabel('D_PCR %d'%(i+1))
    plt.ylabel('A_PCR %d'%(i+1))
    plt.title('D_PCR %d'%(i+1) +'vs A_PCR %d'%(i+1)) 

plt.savefig('/Volumes/E/MAST30034_Python/plots/Q2.5_PCR.pdf')
plt.show()

In [ ]:
Z = U

plt.figure(figsize=(18,10))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.scatter(Z[:,1], std_TC[:,1])
    plt.xlabel('Z %d'%(i+1))
    plt.ylabel('TC %d'%(i+1))
    plt.title('Z %d'%(i+1) +'vs TC %d'%(i+1)) 

plt.savefig('/Volumes/E/MAST30034_Python/plots/Q2.5_ZvsTC.pdf')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as mean_sqr_error

# Compute the MSE of PCR
pred = U @ np.diag(V) @ W
mean_sqr_error(D, pred)

In [ ]:
# Check the distribution for gamma t and gamma s

sns.distplot(gamma_t.flatten(), bins=25, label='temporal')
sns.distplot(gamma_s.flatten(), bins=25, label='spatial')
plt.legend()
plt.title('Data Distribution of Temporal and Spatial Noise')
plt.savefig('/Volumes/E/MAST30034_Python/plots/Q1.4dist.pdf')
plt.show()

In [ ]:
from statistics import mean

nsrcs = 6
x1 = 21
x2 = 21
V = 441
N = 240
rhos = np.arange(0, 1.05, 0.05)

avg_MSE_lst = []
rho_lst = []
   
# Repeat for each rho value
for i in range(len(rhos)):
    rho = rhos[i]
    step = 1/(LA.norm(std_TC @ std_TC.T)*1.1)
    thr = rho *N *step
        
    # Repeat the lasso for 10 times, each time with a new standardized X
    for n in range(10):
        gamma_t = np.random.normal(0, sqrt(0.25), size=(240,6))
        gamma_s = np.random.normal(0, sqrt(0.015), size=(6,441))
        X = (std_TC + gamma_t)@(SM + gamma_s)
        X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
        
        Ao = np.zeros((nsrcs, 1))
        A = np.zeros((nsrcs, 1))
        A_LR = np.zeros((nsrcs, x1*x2)) 
        MSE_lst = []
    
        # Modified R code for LASSO Regression
        for k in range(x1*x2):
            A = Ao + step *(std_TC.T @ (X[:,k].reshape(-1,1) - std_TC @ Ao))
            A_sign = np.sign(A).flatten()
            A = (1/(1+thr))*(A_sign*(np.maximum(np.repeat(0, nsrcs) , (np.abs(A)-thr).T.flatten())))
            A = A.reshape(-1,1)
            
            for j in range(10): 
                Ao = A
                A = Ao + step *(std_TC.T @ (X[:,k].reshape(-1,1) - std_TC @ Ao))
                A_sign = np.sign(A).flatten()
                A = (1/(1+thr))*(A_sign*(np.maximum(np.repeat(0, nsrcs) , (np.abs(A)-thr).T.flatten())))
                A = A.reshape(-1,1)

            A_LR[:,k] = A.flatten()
        
        D_LR = X @ A_LR.T
        mse = np.sum(LA.norm((X - D_LR @ A_LR), ord = 'fro'))/ (N *V)
        MSE_lst.append(mse)
            
    avg_MSE_lst.append(mean(MSE_lst))
    rho_lst.append(rho) 

In [ ]:
# Plot 100 randomly selected sample from X
X_sample = X[np.random.randint(X.shape[0], size=100)]

plt.figure(figsize=(12,8))
plt.plot(X_sample)
plt.ylabel('value',fontsize=16)
plt.title('Sample of Time-sereis Data',fontsize=18)
plt.savefig('/Volumes/E/MAST30034_Python/plots/Q1.5sample.pdf')
plt.show()